<a href="https://colab.research.google.com/github/ded-mikhalych/TK_SMM_LRs/blob/main/%D0%9B%D0%A02_%D0%A1%D0%B0%D1%84%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0_%D0%9F_%D0%94_%2C_%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B5%D0%BD%D0%BA%D0%BE%D0%B2_%D0%9A_%D0%95_%2C_%D0%9C%D0%B8%D1%85%D0%B0%D0%BB%D1%8C%D1%87%D1%83%D0%BA_%D0%94_%D0%90_%2C_%D0%9E%D0%B4%D0%B8%D0%BD_%D0%92_%D0%95_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rnd
import itertools

# Часть 1

## 2.1. Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [ ]:
def init_x_matrix(n, k, d):
    """
    n - число столбцов (длина строки)
    k - число строк
    d - минимальное число единиц в строке -(минус) 1
    """
    x_matr = []
    while (len(x_matr) != k):
        row = [0 for t in range(n)]
        for _ in range(n):
            idx = rnd.randint(0, n - 1)
            row[idx] = 1
            if (is_correct_row(x_matr, row, d) and not row in x_matr):
                x_matr.append(row)
                break
    return np.array(x_matr)

In [ ]:
def is_correct_row(x_matr, row, d):
    X = x_matr.copy()
    X.append(row)
    flag = True
    for i in range(1, d):
        comb = list(itertools.permutations(X, i))
        for c in comb:
            sum_ = np.sum(list(c), axis=0) % 2
            if sum_.tolist().count(1) < (d - i):
                flag = False
                break
        if not flag:
            break
    return flag

In [ ]:
def get_g_matr(x_matr):
    eye_matr = np.eye(x_matr.shape[0], dtype=int)
    return np.concatenate((eye_matr, x_matr), axis=1)

In [ ]:
x_matr = init_x_matrix(3, 4, 3)
g_matr = get_g_matr(x_matr)
g_matr

array([[1, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 1]])

## 2.2 Сформировать проверочную матрицу на основе порождающей

In [ ]:
def get_h_matr(x_matr):
    eye_matr = np.eye(x_matr.shape[1], dtype=int)
    return np.concatenate((x_matr, eye_matr), axis=0)

In [ ]:
h_matr = get_h_matr(x_matr)
h_matr

array([[1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

## 2.3 Сформировать таблицу синдромов для всех однократных ошибок

In [ ]:
def get_syndrome(h_matr, t):
    rows = h_matr.shape[0]
    cols = h_matr.shape[1]
    errors_list = list(np.eye(rows, dtype=int))

    if len(errors_list) != (rows ** t):
        indexes = [i for i in range(rows)]
        combinations_list = list(itertools.combinations(indexes, t))
        t_errors_list = []
        for combination in combinations_list:
            row = np.zeros(rows, dtype=int)
            for idx in combination:
                row[idx] = 1
            t_errors_list.append(np.array(row))
        zeros_matr = [np.zeros(rows, dtype=int) for _ in range(len(t_errors_list))] * t
        for i in range(len(t_errors_list)):
            zeros_matr[i] = t_errors_list[i]
            zeros_matr[i + rows - 1] = t_errors_list[i]
        errors_list.extend(zeros_matr)

        for i in range(len(errors_list) - 1, -1, -1):
            if np.any(errors_list[i]) == 0:
                errors_list = np.delete(errors_list, i, 0)

    errors_list = np.array(errors_list)
    syndroms_list = np.dot(errors_list, h_matr) % 2
    syndrome_dict = dict()
    for i in range(len(syndroms_list)):
        syndrome_dict[tuple(syndroms_list[i])] = errors_list[i]
    return syndrome_dict

In [ ]:
syndrome = get_syndrome(h_matr, 1)

## 2.4 Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова

In [ ]:
def get_err_word(g_matr, h_matr, syndrome, t):
#     если syndrome[tuple(syndrom)] выдаёт ошибку, то тогда просто перегенерируйте все матррицы заново
    row = g_matr.shape[0]
    idx = rnd.randint(0, row - 1)
    u = np.array(g_matr[idx][:row])
    v = np.dot(u, g_matr) % 2
    print(f"Исходное сообщение: {u}")
    print(f"Отправленное сообщение: {v}", )
    v_err = v.copy()
    for i in range(1, t + 1):
        v_err[i] += 1
        v_err[i] %= 2
    print(f"Принятое сообщение с ошибкой: {v_err}")
    print(f"Декодированное сообщение без исправления: {v_err[:row]}")
    syndrom = np.dot(v_err, h_matr) % 2
    print(f"Синдром: {syndrom}")
    error = syndrome[tuple(syndrom)]
    print(f"Ошибка: {error}")
    fixed_message = (v_err + error) % 2
    print(f"Исправленное сообщение: {fixed_message}")
    print(f"Декодированное сообщение: {fixed_message[:row]}")

In [ ]:
get_err_word(g_matr, h_matr, syndrome, 1)

Исходное сообщение: [1 0 0 0]
Отправленное сообщение: [1 0 0 0 1 1 0]
Принятое сообщение с ошибкой: [1 1 0 0 1 1 0]
Декодированное сообщение без исправления: [1 1 0 0]
Синдром: [1 0 1]
Ошибка: [0 1 0 0 0 0 0]
Исправленное сообщение: [1 0 0 0 1 1 0]
Декодированное сообщение: [1 0 0 0]


## 2.5 Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного

In [ ]:
get_err_word(g_matr, h_matr, syndrome, 2)

Исходное сообщение: [1 0 0 0]
Отправленное сообщение: [1 0 0 0 1 1 0]
Принятое сообщение с ошибкой: [1 1 1 0 1 1 0]
Декодированное сообщение без исправления: [1 1 1 0]
Синдром: [1 1 0]
Ошибка: [1 0 0 0 0 0 0]
Исправленное сообщение: [0 1 1 0 1 1 0]
Декодированное сообщение: [0 1 1 0]


# Часть 2

## 2.6. Сформировать порождающую матрицу линейного кода (n, k, 5)

In [ ]:
x_matr2 = init_x_matrix(9, 4, 5)
g_matr2 = get_g_matr(x_matr2)
g_matr2

array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1]])

##  2.7 Сформировать проверочную матрицу на основе порождающей

In [ ]:
h_matr2 = get_h_matr(x_matr2)
h_matr2

array([[0, 1, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]])

##  2.8 Сформировать таблицу синдромов для всех однократных и двукратных ошибок

In [ ]:
syndrome2 = get_syndrome(h_matr2, 2)
syndrome2

{(0, 1, 1, 0, 1, 0, 1, 0, 0): array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (0, 0, 1, 1, 0, 0, 0, 1, 1): array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (0, 0, 0, 0, 1, 1, 1, 1, 0): array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (1, 1, 0, 1, 0, 1, 0, 0, 1): array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (1, 0, 0, 0, 0, 0, 0, 0, 0): array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 (0, 1, 0, 0, 0, 0, 0, 0, 0): array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 (0, 0, 1, 0, 0, 0, 0, 0, 0): array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (0, 0, 0, 1, 0, 0, 0, 0, 0): array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 (0, 0, 0, 0, 1, 0, 0, 0, 0): array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 (0, 0, 0, 0, 0, 1, 0, 0, 0): array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 (0, 0, 0, 0, 0, 0, 1, 0, 0): array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 (0, 0, 0, 0, 0, 0, 0, 1, 0): array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 (0, 0, 0, 0, 0, 0, 0, 0, 1): array([0, 0, 0, 0, 0, 0, 0, 0, 0, 

##  2.9. Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова

In [ ]:
get_err_word(g_matr2, h_matr2, syndrome2, 1)

Исходное сообщение: [0 0 0 1]
Отправленное сообщение: [0 0 0 1 1 1 0 1 0 1 0 0 1]
Принятое сообщение с ошибкой: [0 1 0 1 1 1 0 1 0 1 0 0 1]
Декодированное сообщение без исправления: [0 1 0 1]
Синдром: [0 0 1 1 0 0 0 1 1]
Ошибка: [0 1 0 0 0 0 0 0 0 0 0 0 0]
Исправленное сообщение: [0 0 0 1 1 1 0 1 0 1 0 0 1]
Декодированное сообщение: [0 0 0 1]


##  2.10. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова

In [ ]:
get_err_word(g_matr2, h_matr2, syndrome2, 2)

Исходное сообщение: [0 0 1 0]
Отправленное сообщение: [0 0 1 0 0 0 0 0 1 1 1 1 0]
Принятое сообщение с ошибкой: [0 1 0 0 0 0 0 0 1 1 1 1 0]
Декодированное сообщение без исправления: [0 1 0 0]
Синдром: [0 0 1 1 1 1 1 0 1]
Ошибка: [0 1 1 0 0 0 0 0 0 0 0 0 0]
Исправленное сообщение: [0 0 1 0 0 0 0 0 1 1 1 1 0]
Декодированное сообщение: [0 0 1 0]


##  2.11. Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного

In [ ]:
get_err_word(g_matr2, h_matr2, syndrome2, 3)

Исходное сообщение: [0 0 0 1]
Отправленное сообщение: [0 0 0 1 1 1 0 1 0 1 0 0 1]
Принятое сообщение с ошибкой: [0 1 1 0 1 1 0 1 0 1 0 0 1]
Декодированное сообщение без исправления: [0 1 1 0]
Синдром: [1 1 1 0 1 0 1 0 0]
Ошибка: [1 0 0 0 1 0 0 0 0 0 0 0 0]
Исправленное сообщение: [1 1 1 0 0 1 0 1 0 1 0 0 1]
Декодированное сообщение: [1 1 1 0]
